In [374]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [375]:
import pandas as pd
df = pd.read_csv('https://github.com/FadlyHaikal/ReviewDataset/blob/main/Reviews.csv?raw=true')
df.head()

,Score,Summary,Text
0,5,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,4,"""Delight"" says it all",This is a confection that has been around a fe...
3,2,Cough Medicine,If you are looking for the secret ingredient i...
4,5,Great taffy,Great taffy at a great price. There was a wid...


In [376]:
label = df['Score'].copy()
label.replace({1:0,2:0,3:1,4:1,5:1}, inplace=True)
df["Target"] = label
df.tail(10)
df.drop(['Score', 'Text'], axis=1, inplace=True)

In [416]:
df['Target'].value_counts()

1    5416
0    1026
Name: Target, dtype: int64

In [377]:
df.isna().sum()

Summary    0
Target     0
dtype: int64

In [382]:
df[["Summary", "Target"]]

,Summary,Target
0,Good Quality Dog Food,1
1,Not as Advertised,0
2,"""Delight"" says it all",1
3,Cough Medicine,0
4,Great taffy,1
...,...,...
6437,Will not do without,1
6438,disappointed,0
6439,Perfect for our maltipoo,1
6440,Favorite Training and reward treat,1


In [383]:
import nltk
def punc_clean(text):
    import string as st
    a=[w for w in text if w not in st.punctuation]
    return ''.join(a)
df['Summary'] = df['Summary'].apply(punc_clean)
df.head(2)

,Summary,Target
0,Good Quality Dog Food,1
1,Not as Advertised,0


In [384]:
def remove_stopword(text):
    stopword=nltk.corpus.stopwords.words('english')
    stopword.remove('not')
    a=[w for w in nltk.word_tokenize(text) if w not in stopword]
    return ' '.join(a)
df['Summary'] = df['Summary'].apply(remove_stopword)
df.head(2)

,Summary,Target
0,Good Quality Dog Food,1
1,Not Advertised,0


In [385]:
df['Summary']

0                Good Quality Dog Food
1                       Not Advertised
2                         Delight says
3                       Cough Medicine
4                          Great taffy
                     ...              
6437                  Will not without
6438                      disappointed
6439                  Perfect maltipoo
6440    Favorite Training reward treat
6441                       Great Honey
Name: Summary, Length: 6442, dtype: object

In [386]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectr = TfidfVectorizer(ngram_range=(1,2),min_df=1)
vectr.fit(df['Summary'])
vect_X = vectr.transform(df['Summary'])

In [372]:
X = vect_X
ylabels = df['Target']

In [373]:
print(X)

  (0, 9441)	0.5290964819198507
  (0, 9431)	0.3582064476677744
  (0, 4963)	0.4853558062930075
  (0, 4816)	0.21196353062439557
  (0, 4300)	0.29552783504767677
  (0, 3273)	0.35716337506319135
  (0, 3268)	0.3103931037856348
  (1, 7990)	0.6935561934339715
  (1, 7985)	0.30489156662553946
  (1, 238)	0.652702795420706
  (2, 10055)	0.5826741668455837
  (2, 3048)	0.6308441369721183
  (2, 3046)	0.512373389374105
  (3, 7275)	0.5859424382044479
  (3, 2600)	0.5859424382044479
  (3, 2599)	0.559770415636673
  (4, 11163)	0.6488699059837577
  (4, 5345)	0.7217201422584071
  (4, 5117)	0.24101427627245245
  (5, 11163)	0.605262035927862
  (5, 7884)	0.6732163083001791
  (5, 7846)	0.4247795547141334
  (6, 6319)	0.4005838432406042
  (6, 6306)	0.2662240469570308
  (6, 5253)	0.4193131099055005
  :	:
  (6436, 10705)	0.2907159046959591
  (6436, 6530)	0.37003887770522764
  (6436, 6529)	0.3417834646165505
  (6436, 6252)	0.3326872533106573
  (6436, 5421)	0.37003887770522764
  (6436, 5416)	0.31352805152787344
  (6437,

In [387]:
print(ylabels)

0       1
1       0
2       1
3       0
4       1
       ..
6437    1
6438    0
6439    1
6440    1
6441    1
Name: Target, Length: 6442, dtype: int64


In [388]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2, random_state=0)

In [390]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
clf=model.fit(X_train, y_train)
clf.score(vect_X,df['Target'])*100

87.96957466625271

In [408]:
clf.predict(vectr.transform([""]))

array([1])

In [397]:
clf.predict(vectr.transform(["good"]))

array([1])

#Evaluate

In [409]:
sample_prediction = model.predict(X_test)

In [413]:
from sklearn import metrics
accuracy = metrics.accuracy_score(y_test, sample_prediction)
precision = metrics.precision_score(y_test, sample_prediction)
recall = metrics.recall_score(y_test, sample_prediction)

print(accuracy * 100)
print(precision * 100)
print(recall * 100)

86.42358417377812
86.42570281124497
99.44547134935306


In [415]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, sample_prediction)

array([[  38,  169],
       [   6, 1076]])

In [ ]:
!pip install -q gradio
!pip install -q transformers

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import gradio as gr
from transformers import pipeline

sentiment=pipeline("sentiment-analysis")

def get_sentiment(text):
  txt_split = text.split(";")
  data = {'text': [], 'label': [], 'score': []}
  for t in txt_split:
    res = sentiment(t)
    data["text"].append(t)
    data["label"].append(res[0]['label'])
    data["score"].append(res[0]['score'])

  df = pd.DataFrame(data)
  res = df.groupby('label')['text'].count() / df['label'].count()
  return (res.to_dict(), df)

In [ ]:
iface = gr.Interface(fn=get_sentiment,
                    inputs="text",
                    allow_flagging="never",
                    outputs = ["label", "dataframe"],
                    title="Sentiment Analysis",
                    description='Give the sentiment analysis')

In [ ]:
iface.launch(debug=True)

That's all! Go ahead and open that share link in a new tab. Check out our [getting started](https://gradio.app/getting_started.html) page for more complicated demos.